In [ ]:
import django
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

django.setup()

from dbfcore.dataset.hooks import get_datasources_configured_with_env
from modelbackend.common import (
    get_forecasts_from_furnace_targets_file,
    get_signal_group_data,
    get_blast_furnace_model,
)
from dbfcore.predictionmodel.metrics import SimpleOrAbsoluteErrorMetric
from modelbackend.models import BlastFurnaceModelDefinition, BlastFurnacePredictionResult
from modelfrontend.views import get_blast_furnace_model_result

In [3]:
datasources = get_datasources_configured_with_env()

for furnace_id in [2, 3]:
    df = get_forecasts_from_furnace_targets_file(furnace_id)
    print(f"Number of excel forecasts for BF{furnace_id}", len(df))

Number of excel forecasts for BF2 42592
Number of excel forecasts for BF3 9480


In [4]:
def calculate_metrics_for_excel_forecasts(furnace_ids: list[int], verbose: bool) -> list[pd.DataFrame]:
    output: list[pd.DataFrame] = []
    for furnace_id in furnace_ids:
        target_name = f"bf{furnace_id}_hotmetalsi_chem_pct"
        metric = SimpleOrAbsoluteErrorMetric(
            "simple", furnace_id, target_name, pd.Timedelta("30min"), pd.Timedelta("5min"), verbose=verbose
        )
        forecasts_df = get_forecasts_from_furnace_targets_file(furnace_id)
        calc_times, forecasts = forecasts_df["calc_time"], forecasts_df["forecast_si"]
        data_definition = metric.get_data_definition(calc_times)
        signal_group_data = get_signal_group_data(data_definition, datasources)
        metric_data = metric.calculate_metric_from_excel_predictions(calc_times, forecasts, signal_group_data)
        output.append(metric_data)
    return output

In [5]:
def calculate_metrics_for_model_forecasts(furnace_ids: list[int], verbose: bool) -> list[pd.DataFrame]:
    output: list[pd.DataFrame] = []
    for furnace_id in furnace_ids:
        model_id = 3 if furnace_id == 2 else 1
        target_name = f"bf{furnace_id}_hotmetalsi_chem_pct"
        metric = SimpleOrAbsoluteErrorMetric(
            "simple", furnace_id, target_name, pd.Timedelta("30min"), pd.Timedelta("5min"), verbose=verbose
        )
        calc_times = get_forecasts_from_furnace_targets_file(furnace_id)["calc_time"]
        model_definition = BlastFurnaceModelDefinition.objects.get(id=model_id)

        if f"Silicon model for BF{furnace_id}" not in model_definition.neural_model_def.name:
            raise ValueError(f"Invalid model selected for furnace {furnace_id}")

        model = get_blast_furnace_model(model_definition)
        results_db = BlastFurnacePredictionResult.objects.filter(
            model_definition=model_definition, calc_time__in=calc_times
        )
        results = [get_blast_furnace_model_result(result_db, model) for result_db in results_db]

        data_definition = metric.get_data_definition(calc_times)
        signal_group_data = get_signal_group_data(data_definition, datasources)
        output.append(metric.calculate_metric(calc_times, results, signal_group_data))
    return output

In [6]:
excel_output = calculate_metrics_for_excel_forecasts([2,3], False)
model_output = calculate_metrics_for_model_forecasts([2,3], False)

fig = make_subplots(rows=2, cols=2)

excel_errors_bf2 = excel_output[0].dropna().to_numpy().flatten()
excel_errors_bf3 = excel_output[1].dropna().to_numpy().flatten()
model_errors_bf2 = model_output[0].dropna().to_numpy().flatten()
model_errors_bf3 = model_output[1].dropna().to_numpy().flatten()


fig.add_trace(
    go.Histogram(
        x=excel_errors_bf2,
        name=f"BF2 Excel forecasts error (MAE {round(np.absolute(excel_errors_bf2).mean().item(), 2)})",
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Histogram(
        x=excel_errors_bf3,
        name=f"BF3 Excel forecasts error (MAE {round(np.absolute(excel_errors_bf3).mean().item(), 2)})",
    ),
    row=1,
    col=2,
)
fig.add_trace(
    go.Histogram(
        x=model_errors_bf2,
        name=f"BF2 Model forecasts error (MAE {round(np.absolute(model_errors_bf2).mean().item(), 2)})",
    ),
    row=2,
    col=1,
)
fig.add_trace(
    go.Histogram(
        x=model_errors_bf3,
        name=f"BF3 Model forecasts error (MAE {round(np.absolute(model_errors_bf3).mean().item(), 2)})",
    ),
    row=2,
    col=2,
)


/home/miro/python/DBF/dbfcore/dbfcore/utils.py:38: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  return torch.from_numpy(np.frombuffer(data, dtype=np.float32))


In [ ]:
excel_output = calculate_metrics_for_excel_forecasts([2,3], True)
model_output = calculate_metrics_for_model_forecasts([2,3], True)

excel_output[0].dropna().merge(
    model_output[0].dropna(), left_index=True, right_index=True, suffixes=["_excel", "_model"]
).drop(columns=["real_model"]).reset_index().to_csv("bf2.csv")